In [1]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

In [2]:

images_dir = r"C:\Users\DELL\OneDrive\Desktop\Open Me\chor_police\dataset\images"   
annotations_dir = r"C:\Users\DELL\OneDrive\Desktop\Open Me\chor_police\dataset\annotations"


In [3]:
classes = ["gun", "knife", "smartphone", "credit_card"]

def parse_annotation(xml_file):
    """Parse XML annotation and extract bounding boxes and labels."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    objects = []
    for obj in root.findall("object"):
        label = obj.find("name").text
        if label not in classes:
            continue
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        objects.append((label, xmin, ymin, xmax, ymax))
    return width, height, objects

def load_data(images_dir, annotations_dir):
    """Load data and corresponding labels."""
    images = []
    labels = []

    for xml_file in os.listdir(annotations_dir):
        if not xml_file.endswith(".xml"):
            continue
        xml_path = os.path.join(annotations_dir, xml_file)
        img_file = os.path.splitext(xml_file)[0] + ".jpg"
        img_path = os.path.join(images_dir, img_file)

        if not os.path.exists(img_path):
            continue

        _, _, objects = parse_annotation(xml_path)

        for obj in objects:
            label, _, _, _, _ = obj
            images.append(img_path)
            labels.append(classes.index(label))  
    return images, labels

def preprocess_image(image_path, label, img_height=150, img_width=150):
    """Load and preprocess images."""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  
    return image, label

image_paths, labels = load_data(images_dir, annotations_dir)

image_paths, labels = shuffle(image_paths, labels, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

fold = 1
cv_scores = []

for train_index, val_index in kf.split(image_paths):
    print(f"Training Fold {fold}...")

    train_images = [image_paths[i] for i in train_index]
    val_images = [image_paths[i] for i in val_index]
    train_labels = [labels[i] for i in train_index]
    val_labels = [labels[i] for i in val_index]

    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_dataset = train_dataset.map(preprocess_image).shuffle(500).batch(32)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
    val_dataset = val_dataset.map(preprocess_image).batch(32)

    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(len(classes), activation="softmax")  
    ])

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=10
    )
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Fold {fold} - Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    cv_scores.append(val_accuracy)

    fold += 1

# Print overall results
print(f"Cross-Validation Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# Save the final model (trained on the last fold)


Training Fold 1...
Epoch 1/10


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 25s 838ms/step - accuracy: 0.8477 - loss: 0.2327 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 808ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 852ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 23s 859ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 941ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 922ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 908ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accu

22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 795ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 793ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 804ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 916ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 792ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 787ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 10/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 808ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - accuracy: 

22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 721ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 720ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 730ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 10/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 813ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Fold 9 - Validation Loss: 0.0000, Validation Accuracy: 1.0000
Training Fold 10...
Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 723ms/step - accuracy: 0.8528 - loss: 0.2331 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 714ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━

In [4]:
model.save("object_classifier_cv.h5")

In [5]:
import joblib

with open("object_classifier_cv.pkl", "wb") as file:
    joblib.dump(model, file)